# Define environment

In [ ]:
SYSTEM = "Kaggle" # "Paperspace" # 

# import module

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

# Get Argument

In [ ]:
if SYSTEM == "Kaggle":
    Image_Path = "../input/Train/"
    Crop_Path  = "./TestLargeCrop/JPEGImages/"
else:
    Image_Path = "/home/paperspace/Project/Sealion/TestLargeRaw/"
    Crop_Path  = "/home/paperspace/Project/Sealion/TestLargeCrop/JPEGImages/"

# Define Cropped Image Size

In [ ]:
Crop_Size = (416,416)

# Get Original Images

In [ ]:
file_names = os.listdir(Image_Path)
file_names = sorted(file_names, key=lambda 
                    item: (int(item.partition('.')[0]) if item[0].isdigit() else float('inf'), item))
file_names = file_names[:1]

In [ ]:
print(file_names)

# Crop Images

In [ ]:
def ceil_devide(Big, Small):
    result = int(Big/Small)
    if(Big%Small != 0):
        result += 1
    return result

In [ ]:
def create_crop_template(filename):
    ### remove existing template
    if os.path.exists(Image_Path + 'crop_template.jpg'):
        os.remove(Image_Path + 'crop_template.jpg')
        
    image = cv2.imread(Image_Path + filename)
    image = image[:Crop_Size[1],:Crop_Size[0],:]
    image = cv2.absdiff(image,image)

    cv2.imwrite(Image_Path + 'crop_template.jpg',image)

In [ ]:
def delete_crop_template():
    os.remove(Image_Path + 'crop_template.jpg')

In [ ]:
def create_crop_file():
    if(SYSTEM == "Kaggle"):
        if not os.path.exists("./TestLargeCrop/"):
            os.makedirs("./TestLargeCrop/")
        if not os.path.exists("./TestLargeCrop/JPEGImages/"):
            os.makedirs("./TestLargeCrop/JPEGImages/")        
    else:
        if not os.path.exists("/home/paperspace/Project/Sealion/TestLargeCrop/"):
            os.makedirs("/home/paperspace/Project/Sealion/TestLargeCrop/")
        if not os.path.exists("/home/paperspace/Project/Sealion/TestLargeCrop/JPEGImages/"):
            os.makedirs("/home/paperspace/Project/Sealion/TestLargeCrop/JPEGImages/")

In [ ]:
create_crop_template(file_names[0])
create_crop_file()

for filename in file_names:
    ### skip if file is not image
    if(filename[-3:] != 'jpg'):
        continue
        
    ### read origin image
    ori_image = cv2.imread(Image_Path + filename)
    Shape = ori_image.shape
    X_Len = Shape[1]
    Y_Len = Shape[0]
    
    X_Amt = ceil_devide(X_Len, Crop_Size[0])
    Y_Amt = ceil_devide(Y_Len, Crop_Size[1])
    
    cnt = 0
    for j in range(Y_Amt):
        for i in range(X_Amt):
            # counting
            cnt += 1
            
            # create crop image
            crop_image = cv2.imread(Image_Path + 'crop_template.jpg')
            crop_image = ori_image[j*Crop_Size[1]:(j+1)*Crop_Size[1], i*Crop_Size[0]:(i+1)*Crop_Size[0], :]
            
            # save crop image
            Name = Crop_Path + filename.split('.')[0] + '_' + str(cnt) + '.jpg'
            cv2.imwrite(Name, crop_image) 
    
delete_crop_template()